In [9]:
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dlel2\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [8]:
import nltk
from nltk.corpus import stopwords

In [11]:
stop = stopwords.open('english').read()

In [12]:
s = "i'd like to learn more somthing. i'd like to learn more somthin."

In [13]:
from nltk.tokenize import word_tokenize

In [14]:
word_tokenize(s)[1] in stop.split()

False

In [20]:
import re
from string import punctuation

result = list()
for t in word_tokenize(
    re.sub('[{}]'.format(re.escape(punctuation)),
           '', 
           'To be or not to be'.lower())):
    if t in stop.split():
        print(t, 'skipped')
    else:
        result.append(t)
        print(t)
' '.join(result)

to skipped
be skipped
or skipped
not skipped
to skipped
be skipped


''

In [24]:
# 한글 불용어 사전 - 존재는 하나 영어처럼 표준은 없다.
from konlpy.tag import Komoran


# ma = Komoran()
stop = ['은', '는', '이', '가', '을', '를', '께서', '에게', '다고', '시', '었어', '다']

result = list()
for t in word_tokenize(
    re.sub('[{}]'.format(re.escape(punctuation)),
           '', 
           '어머니는 짜장면이 싫다고 하셨어.')): # 토큰화
    if t in stop:
        print(t, 'skipped')
    else:
        result.append(t)
        print(t)
' '.join(result)

어머니는
짜장면이
싫다고
하셨어


'어머니는 짜장면이 싫다고 하셨어'

In [30]:
d = '''어머니는 짜장면이 핵노맛이래.
어머니는 짜장면이 싫다고 하셨어.'''
for s in d.splitlines():
    print([t[0] for t in ma.pos(s) if t[1][0] not in ['E', 'J']])

['어머니', '짜장면', '핵', '노', '맛', '이', '.']
['어머니', '짜장면', '싫', '하', '.']


# 욕설제거

In [39]:
stop = ['시발', '씨발', '시발놈', '씨발놈']
result = list()

for t in word_tokenize(
    re.sub('[{}]'.format(re.escape(punctuation)), '',
           '어쩌고 저쩌고 씨이ㅣ발!!!')):
    if t in stop:
        result.append('*'*len(t))
    else:
        result.append(t)
' '.join(result)

'어쩌고 저쩌고 씨이ㅣ발'

In [34]:
ma.pos('어쩌고 저쩌고 씨발놈')

[('어쩌', 'VV'), ('고', 'EC'), ('어쩌', 'VV'), ('고', 'EC'), ('씨발놈', 'NNG')]

In [55]:
D = {'시발':5,'시이발':2,'C발':5,'씨발':3}

def preprocessing(d):
    result = dict()
    for k,v in d.items():
        nk = ' '.join(['<w>',]+list(k)+['</w>',])
        result[nk] = v
    return result


p = list()
d = preprocessing(D)

for _ in range(5):
    pairs = dict()
    for k,v in d.items():
        for i in range(len(k.split())-1):
            bigram = ' '.join(k.split()[i:i+2])
            if bigram in pairs:
                pairs[bigram] += v
            else:
                pairs[bigram] = v
    bestkey = max(pairs, key=pairs.get)
    newd = dict()
    for k,v in d.items():
        nk = re.sub(bestkey, re.sub(' ', '', bestkey), k)
        newd[nk] = v
    d = newd
    print(re.sub(' ', '', bestkey))
    p.append(bestkey)

발</w>
<w>시
<w>시발</w>
<w>C
<w>C발</w>


In [50]:
s = list()
e = list()

for t in p:
    if t.split()[0] == '<w>':
        s.append(''.join(t.split()[1:]))
    if t.split()[-1] == '</w>':
        e.append(''.join(t.split()[:-1]))

In [51]:
pattern = list()
for i in s:
    for j in e:
        pattern.append(re.compile(r'\b{}.*?{}\b'.format(i,j)))

In [52]:
pattern

[re.compile(r'\b시.*?발\b', re.UNICODE), re.compile(r'\bC.*?발\b', re.UNICODE)]

In [54]:
for d in D:
    if sum([p.search(d) != None for p in pattern]) > 0:
        print('*'*len(d))
        
    else:
        print(d)

**
***
**
씨발


In [57]:
chr(ord('가')+29)

'객'

In [58]:
ord('가'), chr(ord('가'+30))

TypeError: can only concatenate str (not "int") to str

In [59]:
base = ord('가')

In [60]:
for i in range(10):
    print(chr(base+(28*21)+i))

까
깍
깎
깏
깐
깑
깒
깓
깔
깕


In [61]:
19*21*28

11172

In [ ]:
i 

In [64]:
def str2trigem(s):
    base = ord('가')
    result = list()
    for c in list(s):
        if re.match('[가-힣]', c):
            c1, c23 = divmod(ord(c)-base, 21*28)
            c2, c3 = divmod(c23, 28)
            result.append(cho[c1])
            result.append(jung[c2])
            result.append(jong[c3])
        else:
            result.append(c)
    return ''.join(result)

def trigem2str(s):
    result = list()
    i = 0
    while True:
        
#     for i in range(len(s)):
        if re.match('[{}]'.format(''.join(cho)), s[i]) and \
            re.match('[{}][{}][{}]'.format(
            ''.join(cho), ''.join(jung), ''.join(jong)), s[i:i+3]):
        
            c1 = cho.index(s[i])
            c2 = jung.index(s[i+1])
            c3 = jong.index(s[i+2])
            c = (28+21)*c1
            c += 28*c2
            c += c3
            result.append(chr(ord('가')+c))
            
            i += 3
        else:
            result.append(s[i])
            i += 1
        
        if i >= len(s):
            break
            
    return ''.join(result)
# str2trigem('동해물과 백두산이')
trigem2str('ㄷㅗㅇㅎㅐ ㅁㅜㄹㄱㅘ ')

In [65]:
str2trigem('동해물과 백두산이')

NameError: name 'cho' is not defined

In [66]:
divmod(30,7) # 몫과 나머지 튜플 (몫, 나머지)

(4, 2)

In [69]:
def hammingDist(s1, s2, useTrigem=False):
    if len(s1) != len(s2):
        return '길이가 다름'
    
    if useTrigem:
        t1, t2 = str2trigem(s1), str2trigem(s2)
        
    return sum([c1 != c2 for c1, c2 in zip(s1, s2)])

hammingDist('고려대학교', '고구려대학교'),\
hammingDist('고려대학교', '고구려대학'),\
hammingDist('고려대학교', '골려대학교'),\
hammingDist('고려대학교', '골려대학교', True)

NameError: name 'cho' is not defined

In [ ]:
def lev(s1, s2):
    if len(s1) == 0:                 # case 1
        return len(s2)
    if len(s2) == 0:                 # case 2
        return len(s1)
    if s1[0] == s2[0]:               # case 3
        return lev(s1[1:], s2[1:])
    else:
        return min([
            1+ lev(s1[1:], s2),      # case4
            1+ lev(s1, s2[1:]),      # case 5
            1+ lev(s1[1:], s2[1:])   # case 6
        ])